In [123]:
# Import needed modules
import pandas as pd # type: ignore
import subprocess

In [124]:
# Load the data
path_csv = "./negative_noCDS_relationship/blastn_neg_noMatchCds_plus100nt.csv"
df = pd.read_csv(path_csv, sep=",", header=0)
# df.reset_index(drop=True, inplace=True)
print(df.shape)
df.head()
print(df.dtypes)
df.head()

(2327, 14)
qseqid       object
sseqid       object
pident      float64
length        int64
mismatch      int64
gapopen       int64
qstart        int64
qend          int64
sstart        int64
send          int64
evalue      float64
bitscore    float64
qlen          int64
slen          int64
dtype: object


,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,qlen,slen
0,Seq_0_LinJ.01,Seq_0_LinJ.01,100.0,140,0,0,1,140,1,140,2.720000e-71,259.0,140,140
1,Seq_1_LinJ.01,Seq_1_LinJ.01,100.0,629,0,0,1,629,1,629,0.000000e+00,1162.0,629,629
2,Seq_2_LinJ.01,Seq_2_LinJ.01,100.0,119,0,0,1,119,1,119,1.060000e-59,220.0,119,119
3,Seq_3_LinJ.01,Seq_3_LinJ.01,100.0,574,0,0,1,574,1,574,0.000000e+00,1061.0,574,574
4,Seq_4_LinJ.01,Seq_4_LinJ.01,100.0,288,0,0,1,288,1,288,3.180000e-153,532.0,288,288


In [125]:
# Let's see the cunt values in the "qseqid"
df["sseqid"].value_counts()

sseqid
Seq_266_LinJ.22    368
Seq_5_LinJ.01      127
Seq_265_LinJ.22    100
Seq_560_LinJ.34     15
Seq_566_LinJ.34     15
                  ... 
Seq_313_LinJ.25      1
Seq_314_LinJ.26      1
Seq_315_LinJ.26      1
Seq_316_LinJ.26      1
Seq_317_LinJ.26      1
Name: count, Length: 721, dtype: int64

In [126]:
# Since there are some elements that have only one coun value, which is for sure with himself, let's remove the rows where "qseqid" and "sseqid" are the same
df = df[df["qseqid"] != df["sseqid"]]
print(df.shape)
df.head()

(1410, 14)


,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,qlen,slen
24,Seq_5_LinJ.01,Seq_266_LinJ.22,94.069,2664,76,58,412,3025,1,2632,0.0,3969.0,3406,24324
25,Seq_5_LinJ.01,Seq_266_LinJ.22,93.548,2666,92,60,424,3045,557,3186,0.0,3897.0,3406,24324
26,Seq_5_LinJ.01,Seq_266_LinJ.22,93.580,2648,94,54,414,3025,17616,20223,0.0,3879.0,3406,24324
27,Seq_5_LinJ.01,Seq_266_LinJ.22,93.594,2638,96,49,424,3025,21350,23950,0.0,3868.0,3406,24324
28,Seq_5_LinJ.01,Seq_266_LinJ.22,93.530,2643,95,51,424,3025,20816,23423,0.0,3864.0,3406,24324


In [127]:
# Let's count values again
df["sseqid"].value_counts()

sseqid
Seq_266_LinJ.22    189
Seq_5_LinJ.01      108
Seq_265_LinJ.22     99
Seq_566_LinJ.34     14
Seq_560_LinJ.34     14
                  ... 
Seq_32_LinJ.05       1
Seq_720_LinJ.36      1
Seq_715_LinJ.36      1
Seq_717_LinJ.36      1
Seq_642_LinJ.35      1
Name: count, Length: 341, dtype: int64

In [128]:
# Let's check if there are values in "sstart" > "send"
print(sum(df["sstart"] > df["send"]))
df[df["sstart"] > df["send"]].head()

68


,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,qlen,slen
154,Seq_20_LinJ.04,Seq_21_LinJ.04,99.281,278,2,0,10,287,282,5,6.170000e-144,503.0,690,676
155,Seq_20_LinJ.04,Seq_24_LinJ.04,87.019,208,13,3,97,290,222,15,1.940000e-59,222.0,690,222
157,Seq_21_LinJ.04,Seq_20_LinJ.04,99.281,278,2,0,5,282,287,10,6.040000e-144,503.0,676,690
163,Seq_24_LinJ.04,Seq_20_LinJ.04,87.624,202,11,3,21,222,284,97,5.900000e-60,222.0,222,690
176,Seq_34_LinJ.05,Seq_35_LinJ.05,98.039,357,5,1,13,367,357,1,5.870000e-179,619.0,689,685


In [129]:
# Change the values so "sstart" is always < "send"
df.loc[
    df["sstart"] > df["send"],
    ["sstart", "send"]
] = df.loc[
    df["sstart"] > df["send"],
    ["send", "sstart"]
].values

# Let's check if there are values in "sstart" > "send"
print(sum(df["sstart"] > df["send"]))
df[df["sstart"] > df["send"]].head()

0


,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,qlen,slen


In [130]:
# Let's make a "groupby" object
grouped_df = df.groupby("sseqid")
print(f"grouped_df has {len(grouped_df)} groups")

grouped_df has 341 groups


In [131]:
neg_dict = {}
for index,(group_name, group_data) in enumerate(grouped_df, start=1):
    # print(f"index: {index}; group: {group_name}")
    min_sstart = int(group_data["sstart"].min())
    max_send = int(group_data["send"].max())
    slen = group_data["slen"].max()
    number_id = int(group_name.split("_")[1])
    print("="*50)
    print(
        f"For group {group_name}: \n",
        "--------------------------------------\n",
        f"| OG start: 1    ==>    NEW start: {min_sstart} |\n",
        f"| OG end: {slen}    ==>    NEW end: {max_send} |\n"
        "--------------------------------------\n\n",        
        sep=""
        )
    neg_dict[number_id] = {
        "full_name": group_name,
        "start": min_sstart,
        "end": max_send,
    }

For group Seq_100_LinJ.09: 
--------------------------------------
| OG start: 1    ==>    NEW start: 1 |
| OG end: 333    ==>    NEW end: 309 |
--------------------------------------


For group Seq_104_LinJ.10: 
--------------------------------------
| OG start: 1    ==>    NEW start: 1 |
| OG end: 1009    ==>    NEW end: 795 |
--------------------------------------


For group Seq_119_LinJ.12: 
--------------------------------------
| OG start: 1    ==>    NEW start: 1 |
| OG end: 256    ==>    NEW end: 256 |
--------------------------------------


For group Seq_120_LinJ.12: 
--------------------------------------
| OG start: 1    ==>    NEW start: 1 |
| OG end: 607    ==>    NEW end: 523 |
--------------------------------------


For group Seq_121_LinJ.12: 
--------------------------------------
| OG start: 1    ==>    NEW start: 1 |
| OG end: 258    ==>    NEW end: 258 |
--------------------------------------


For group Seq_122_LinJ.12: 
--------------------------------------
| 

In [132]:
# Let's check the dictionary
print(len(neg_dict))
for key, value in neg_dict.items(): print(key, value)

341
100 {'full_name': 'Seq_100_LinJ.09', 'start': 1, 'end': 309}
104 {'full_name': 'Seq_104_LinJ.10', 'start': 1, 'end': 795}
119 {'full_name': 'Seq_119_LinJ.12', 'start': 1, 'end': 256}
120 {'full_name': 'Seq_120_LinJ.12', 'start': 1, 'end': 523}
121 {'full_name': 'Seq_121_LinJ.12', 'start': 1, 'end': 258}
122 {'full_name': 'Seq_122_LinJ.12', 'start': 1, 'end': 523}
124 {'full_name': 'Seq_124_LinJ.12', 'start': 1, 'end': 531}
126 {'full_name': 'Seq_126_LinJ.12', 'start': 1, 'end': 724}
127 {'full_name': 'Seq_127_LinJ.12', 'start': 1, 'end': 725}
128 {'full_name': 'Seq_128_LinJ.12', 'start': 2, 'end': 750}
12 {'full_name': 'Seq_12_LinJ.02', 'start': 1, 'end': 323}
130 {'full_name': 'Seq_130_LinJ.12', 'start': 1, 'end': 530}
131 {'full_name': 'Seq_131_LinJ.12', 'start': 6, 'end': 753}
135 {'full_name': 'Seq_135_LinJ.12', 'start': 1, 'end': 561}
136 {'full_name': 'Seq_136_LinJ.12', 'start': 1, 'end': 184}
138 {'full_name': 'Seq_138_LinJ.12', 'start': 1, 'end': 184}
140 {'full_name': 'Seq

In [133]:
# Sort the dictionary by the "number_id"
neg_dict = dict(sorted(neg_dict.items(), key=lambda x: x[0]))

# Let's check the dictionary
for key, value in neg_dict.items(): print(key, value)

5 {'full_name': 'Seq_5_LinJ.01', 'start': 412, 'end': 3399}
8 {'full_name': 'Seq_8_LinJ.02', 'start': 1, 'end': 323}
12 {'full_name': 'Seq_12_LinJ.02', 'start': 1, 'end': 323}
15 {'full_name': 'Seq_15_LinJ.02', 'start': 1, 'end': 156}
19 {'full_name': 'Seq_19_LinJ.04', 'start': 4, 'end': 498}
20 {'full_name': 'Seq_20_LinJ.04', 'start': 10, 'end': 287}
21 {'full_name': 'Seq_21_LinJ.04', 'start': 1, 'end': 282}
24 {'full_name': 'Seq_24_LinJ.04', 'start': 14, 'end': 222}
30 {'full_name': 'Seq_30_LinJ.05', 'start': 1, 'end': 210}
32 {'full_name': 'Seq_32_LinJ.05', 'start': 1, 'end': 207}
34 {'full_name': 'Seq_34_LinJ.05', 'start': 13, 'end': 364}
35 {'full_name': 'Seq_35_LinJ.05', 'start': 1, 'end': 357}
37 {'full_name': 'Seq_37_LinJ.05', 'start': 1, 'end': 204}
45 {'full_name': 'Seq_45_LinJ.07', 'start': 1, 'end': 1545}
46 {'full_name': 'Seq_46_LinJ.07', 'start': 1, 'end': 196}
47 {'full_name': 'Seq_47_LinJ.07', 'start': 1, 'end': 234}
48 {'full_name': 'Seq_48_LinJ.07', 'start': 1, 'end':

In [134]:
# Load the original csv file:
path_og_csv = "./data/negative_database_nomatch.csv"

og_csv = pd.read_csv(path_og_csv, sep=",", header=0)
print(og_csv.shape)
print(og_csv.dtypes)
og_csv.head(10)

(723, 5)
sseqid     object
sstart      int64
send        int64
sstrand    object
sseq       object
dtype: object


,sseqid,sstart,send,sstrand,sseq
0,LinJ.01,36103,36242,plus,AGACAGACCGACACACGCAGCCGTGTGATGCCGCCGCCGAGGGCAG...
1,LinJ.01,113760,114388,plus,CAGCGCCATGCACGACATGGCCGCTGACGTCCGTAGCCCTAACTCG...
2,LinJ.01,146412,146530,plus,GCGAATTGTGTTCTGCGCATGCCTCTTCTCTGCCGTGCAGCATGCG...
3,LinJ.01,261866,262439,plus,CGGACTTGGCAAGTGGCCGCCATCGATGAAAACGCACCATGCCTTT...
4,LinJ.01,271363,271650,plus,CGAACGCCGCCCTCAATCGCGCGCTGAACTTCACGCGGCGGTCGAC...
5,LinJ.01,272496,275901,plus,GTCAGAACAACGAAGCCGCCCCCCATCACTGCCTCCCTCTACGTAC...
6,LinJ.02,17013,17129,plus,GGGAGGGGACGCGCGGAGCTGCGATGCGAAAGTGAGAGCAACACAG...
7,LinJ.02,74329,74459,plus,GTGTCACAAGCGACTCGAAGAGGACGAAAAGACACACGGCCGCACA...
8,LinJ.02,95421,95743,plus,TCCGCGATCCGTGCAGTTGGCGCCGGCCCCTCCTTCACTGCCGATG...
9,LinJ.02,96677,97128,plus,GTCCGGCTATCTTGCTGTGCTCGGCACTCAGCGAGCTCACTGCACT...


In [135]:
# Backup "og_csv"
og_csv_bk = og_csv.copy()

In [136]:
# Needed function for the next step
def get_sequence(start_coor, end_coor, strand, chromosome, path_genome):
    cmd = f'blastdbcmd -db {path_genome} -entry {chromosome} -range {start_coor}-{end_coor} -strand {strand} -outfmt %s'
    sequence = subprocess.run(cmd, shell=True, capture_output=True, text=True, universal_newlines=True, executable='/usr/bin/bash')
    sequence = sequence.stdout.strip()
    return sequence

In [137]:
for key, value in neg_dict.items():
    og_row = og_csv.iloc[key].copy()
    new_start = og_row["sstart"] + value["start"] - 1
    new_end = og_row["send"] + value["end"] - 1
    
    new_seq = get_sequence(start_coor=new_start,
                           end_coor=new_end,
                           strand=og_row["sstrand"],
                           chromosome=og_row["sseqid"],
                           path_genome="../0.Data/genome/L_infantum/TriTrypDB-67_LinfantumJPCM5_Genome.fasta")
    
    # Update the copy
    og_row["sstart"] = new_start
    og_row["send"] = new_end
    og_row["sseq"] = new_seq
    
    # Replace the row in the original DataFrame
    og_csv.iloc[key] = og_row

In [138]:
# Save database
print(og_csv.shape)
print(og_csv.dtypes)
og_csv.to_csv("./negative_noCDS_relationship/negative_database_nomatch_corrected.csv", sep=",", index=False, header=True)

(723, 5)
sseqid     object
sstart      int64
send        int64
sstrand    object
sseq       object
dtype: object
